## ==**This notebook is under active development**==

# Model Evaluation

In this notebook you will learn:

- What is model evaluation?
- What are common evaluation metrics?
- How to evaluate a model in ML.NET
- Train and evaluate models with cross-validation
- Model explainability
- How to improve your models

## What is model evaluation?

Training is the process of applying algorithms to historical data in order to create a model that accurately represents that data. That model is then used on new data to make predictions. 

To determine  

## What are common evaluation metrics?

## How to evaluate a model in ML.NET

### Reference ML.NET Daily Build Feed

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json

### Install dependencies

In [1]:
#r "nuget:Microsoft.Data.Analysis,0.20.0-preview.22226.2"
#r "nuget:Microsoft.ML.AutoML,0.20.0-preview.22226.2"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22226.2 Microsoft.ML.AutoML, 0.20.0-preview.22226.2

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Reference packages with `using` statements

In [1]:
using System.Text.Json;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;
using static Microsoft.ML.Transforms.OneHotEncodingEstimator;

### Load your data

Use the `#!value` and `#!share` magic commands to fetch the data from GitHub, store it in the `taxi_data` variable and load it into a `DataFrame` 

In [1]:
#!value --name taxi_data --from-url https://github.com/dotnet/csharp-notebooks/raw/main/machine-learning/data/taxi-fare.csv

In [1]:
#!share taxi_data --from value

In [1]:
var df = DataFrame.LoadCsvFromString(taxi_data);

Once the data is loaded, use the `Head` method to preview the first five rows.

In [1]:
df.Head(5)

index,vendor_id,rate_code,passenger_count,trip_time_in_secs,trip_distance,payment_type,fare_amount
0,CMT,1,1,1271,3.8,CRD,17.5
1,CMT,1,1,474,1.5,CRD,8
2,CMT,1,1,637,1.4,CRD,8.5
3,CMT,1,1,181,0.6,CSH,4.5
4,CMT,1,1,661,1.1,CRD,8.5


### Initialize MLContext

All ML.NET operations start in the [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext) class. Initializing mlContext creates a new ML.NET environment that can be shared across the model creation workflow objects. It's similar, conceptually, to DBContext in Entity Framework.

In [1]:
var mlContext = new MLContext();

### Split the data into train and test sets

The train set is what you'll use to learn the patterns of your data. The test set is the dataset used to evaluate the performance of your model using evaluation metrics for the regression task. 

In this case, 80% of the data is used for training and 20% is used for test/evaluation as defined by the `testFraction` parameter.

In [1]:
var trainTestData = mlContext.Data.TrainTestSplit(df,testFraction:0.2);
var validationTestData = mlContext.Data.TrainTestSplit(trainTestData.TestSet,testFraction:0.5);

In [1]:
var trainSet = trainTestData.TrainSet;
var validationSet = validationTestData.TrainSet;
var testSet = validationTestData.TestSet;

### Create training pipeline

For ths dataset, the following transforms are applie:

- OneHotEncoding to convert categorical values into numerical values
- ReplaceMIssingValues which as the name suggests is to replace any missing values.
- Concatenate takes all of the features and creates a feature vector

AutoML is used to define a regression experiment using the `fare_amount` column as the column to predict or label column. 

In [1]:
var pipeline = 
	mlContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(mlContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(mlContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }))
        .Append(mlContext.Auto().Regression(labelColumnName: "fare_amount"));

### Configure experiment

Use AutoML to configure our experiment to train for 60 seconds using the pipeline you've just defined. 

By default, AutoML evaluates the models it trains using the evaluation metric you want to optimize. In this case it's R-Squared which is calculated by comaring the actual value `fare_amount` against the predicted value `Score`. 

In [1]:
var experiment = 
	mlContext.Auto().CreateExperiment()
		.SetPipeline(pipeline)
        .SetTrainingTimeInSeconds(60)
        .SetDataset(trainSet, validationSet)
        .SetEvaluateMetric(RegressionMetric.RSquared, "fare_amount", "Score");

### Run experiment

In [1]:
var result = await experiment.Run();

### View evaluation metrics of best model

In [1]:
$"R-Squared: {result.Metric}"

R-Squared: 0.9095435852083756

Note that during training, the evaluation metrics were calculated using the validation set. To see how well your model performs on new data, evaluate its performance against the test set. 

Start by getting the best model using the `Model` property from the training results. Then, use the `Transform` method to use the model to make predictions against the test dataset.

In [1]:
ITransformer bestModel = result.Model;
var predictions = bestModel.Transform(testSet);

Inspect the first few predictions (`Score` column) and compare them against the actual value (`fare_amount` column). Then, calculate the difference between them.

In [1]:
var actual = predictions.GetColumn<float>("fare_amount");
var predicted = predictions.GetColumn<float>("Score");

var compare = 
	actual
		.Zip(predicted,(actual,pred) => new {Actual=actual, Predicted=pred, Difference=actual-pred})
		.Take(5);

compare

index,Actual,Predicted,Difference
0,24.5,22.489357,2.010643
1,9.5,8.760716,0.73928356
2,4.5,5.5181503,-1.0181503
3,8,8.760716,-0.76071644
4,52,52.465206,-0.46520615


Just by quickly comparing the first few values you can see the predictions are generally just a few cents off from the actual amount. 

With ML.NET, you don't have to manually calculate the evaluation metrics for your models. ML.NET provides a built-in `Evaluate` method for each of the machine learning tasks it supports.  Use the `Evaluate` method for the regression task to calculate the evaluation metrics for the test set where the `fare_amount` column is the actual value and the `Score` column is te predicted value.

In [1]:
var evaluationMetrics = mlContext.Regression.Evaluate(predictions,"fare_amount", "Score");

Using the `Evaluate` method not only calculates the metric you optimized for during training (R-Squared), but also all the metrics for the regression task. 

In [1]:
evaluationMetrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0.9942549315384456,8.621650263904534,2.9362646787891133,8.621650257944713,0.9066165113489004


### Using metrics to evaluate models

TO-DO: Explain Evaluation Metrics

## Train and evaluate models using cross-validation

### What is cross-validation?

Cross-validation is a training and model evaluation technique that splits the data into several partitions and trains multiple models on these partitions. This technique improves the robustness of the model by holding out data from the training process. In addition to improving performance on unseen observations, in data-constrained environments it can be an effective tool for training models with a smaller dataset.

### Train a model using cross-validation

Start off by initializing the `MLContext`. 

In [1]:
var cvMLContext = new MLContext();

Then, define your pipeline. In this case, the actual trainer is used instead of the `SweepableEstimator` from AutoML.

In [1]:
var cvMLPipeline = 
	cvMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(cvMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(cvMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }))
		.Append(cvMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount"));

Use the `CrossValidate` method to start the training and evaluation on your data using the defined pipeline. By default, data is split into five subsets but you can set this to any value you prefer using the `numberOfFolds` parameter.

In [1]:
var cvResults = cvMLContext.Regression.CrossValidate(trainSet, cvMLPipeline, labelColumnName: "fare_amount");

In [1]:
cvResults.Select(x => x.Metrics)

index,MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0,1.4827485647771026,11.963041452547971,3.4587629945614906,11.963041604103823,0.8720854037232295
1,1.2375841321457335,9.705592823696339,3.1153800448254043,9.705592741963914,0.8966878554543226
2,1.2889469436427767,8.062997905014296,2.8395418477307737,8.062997855546088,0.9103292917917389
3,1.4536211884205645,10.900438423862004,3.3015812005555767,10.900438445337578,0.8814025459794065
4,1.221032141380664,8.696668332553719,2.9490114161450305,8.696668288676296,0.9040812184308328


In [1]:
var cvTestEvalMetrics = 
	cvResults
		.Select(fold => fold.Model.Transform(testSet))
		.Select(predictions => cvMLContext.Regression.Evaluate(predictions, "fare_amount", "Score"));

In [1]:
cvTestEvalMetrics

index,MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0,1.4710497047764914,11.568454485871067,3.4012430795035904,11.56845458503681,0.874698856352953
1,1.2358056159405482,9.908180408259156,3.1477262282891054,9.908180533672343,0.8926817460247235
2,1.2984243246192024,10.056430381877949,3.1711875349587806,10.056430374682195,0.8910760093843826
3,1.4455370180379776,11.06478393768647,3.326376998730972,11.064783904037112,0.8801542519536311
4,1.2200094360419682,9.984998343437963,3.15990479974286,9.98499836559539,0.891849709632804


## Model explainability

### Explain models with Permutation Feature Importance (PFI)

### Explain models with Feature Contribution Calculation (FCC)

In [1]:
var fccMLContext = new MLContext();

In [1]:
var fccDataPipeline = 
	fccMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(fccMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(fccMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

In [1]:
var fccTrainer = fccMLContext.Regression.Trainers.Sdca(labelColumnName: "fare_amount");

In [1]:
var transformedData = fccDataPipeline.Fit(trainSet).Transform(trainSet);
var fccModel = fccTrainer.Fit(transformedData);

In [1]:
var fccPredictions = fccModel.Transform(transformedData); 

In [1]:
var fcc = 
	fccMLContext.Transforms.CalculateFeatureContribution(fccModel,normalize:false).Fit(fccPredictions);

In [1]:
var contributions = fcc.Transform(fccPredictions);

#### Get slot names

In [1]:
var featureContributionColumn = contributions.Schema.GetColumnOrNull("FeatureContributions");
var slotNames = new VBuffer<ReadOnlyMemory<char>>();
column.Value.GetSlotNames(ref slotNames)
var slotNameValues = slotNames.DenseValues();

In [1]:
var featureContributionValues = contributions.GetColumn<float[]>("FeatureContributions");

In [1]:
var contribMapping = 
	contributionCalc
		.Select(x => x.Zip(dv,(a,b) => new KeyValuePair<string,float>(b.ToString(),a)));

In [1]:
contribMapping.First()

index,Key,Value
0,vendor_id.Bit2,0
1,vendor_id.Bit1,0
2,vendor_id.Bit0,0
3,payment_type.Bit3,0
4,payment_type.Bit2,0
5,payment_type.Bit1,0
6,payment_type.Bit0,0
7,rate_code,8.500199
8,passenger_count,-0.54379815
9,trip_time_in_secs,8.0872555


## How can I improve my model?

## Continue learning

> [⏪ Last Module - Training and AutoML](https://raw.githubusercontent.com/JakeRadMSFT/csharp-notebooks/main/machine-learning/03-Training%20and%20AutoML.ipynb)

### More End to End Examples

- [Binary Classification with Titanic Dataset](https://raw.githubusercontent.com/JakeRadMSFT/csharp-notebooks/main/machine-learning/E2E-Binary%20Classification%20with%20Titanic%20Dataset.ipynb)  
- [Value Prediction(Regression) with Taxi Dataset](https://raw.githubusercontent.com/JakeRadMSFT/csharp-notebooks/main/machine-learning/E2E-Value%20Prediction(Regression)%20with%20Taxi%20Dataset.ipynb)  
